In [ ]:
import ipywidgets as wdg
from aitw_aiida_viscosity.workflows.viscosity import launch_workflow

import utilities

In [ ]:
import rich

console = rich.get_console()
console.is_jupyter = True
console.width = 160

widgets = utilities.get_widgets_from_click_function(launch_workflow)

title = wdg.HTML(value="<h2>Microstructure Generation Parameters</h2>")
display(title)
widget_box = wdg.VBox(list(widgets.values()))
display(widget_box)

generate_button = wdg.Button(description="Generate Microstructure", button_style="success")
output_area = wdg.Output()

def on_generate_button_clicked(b):
    with output_area:
        output_area.clear_output()
        args = utilities.get_click_args_from_widgets(launch_workflow, widgets)
        launch_workflow(args, standalone_mode=False)

generate_button.on_click(on_generate_button_clicked)
display(generate_button)
display(output_area)